<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.875·Deep Learning · Práctica: Implementación de un algoritmo para la clasificación de enfermedades foliares del árbol del manzano</p>
<p style="margin: 0; text-align:right;">2020-2 · Máster Universitario en Ciencia de Datos (MUDS)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Iñaki Galech Amillano-MSDS 2020/21 <a href="mailto:igalech@uoc.edu">igalech@uoc.edu</a></p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Nombre y apellidos:</strong> Iñaki Galech Amillano - MSDS 2020/21  <a href="mailto:igalech@uoc.edu">igalech@uoc.edu</a>
</div>

Importación de las librerías necesarias. **Nota:** En Colab de google es necesario importar la librería Kaggle por separado.

In [ ]:
%matplotlib inline
import cv2
import glob
import os
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import keras_preprocessing
from keras.preprocessing.image import ImageDataGenerator  
from keras.layers import (
    GlobalAveragePooling2D, Multiply, Flatten,
     Dense, Dropout, Conv2D, BatchNormalization, MaxPooling2D)
from keras.layers.experimental.preprocessing import Resizing, Rescaling
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras import Sequential, Model
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
import albumentations
import shutil

# <u>Preprocesado de los datos y análisis preliminar</u>

Situamos el path en nuestro set de datos, en concreto en el subidectorio /train.

In [ ]:
pathimagenestrain = '/kaggle/input/kaggle-plant-pathology-2021-modificat/train/'

print("El directorio base entrenamiento es {}".format(pathimagenestrain))

### Conteo inicial por subdirectorios
Recorremos los subdirectorios y contamos el número de imágenes por directorio para obtener el conteo por clases.

In [ ]:
# Recorremos los subdirectorios y contamos el número de imágenes por clase

subdirectorios = [x[1] for x in os.walk(pathimagenestrain)]
subdirectorios = subdirectorios[0]

# Diccionario para la cuenta
numeroimagenes = {}

# Recorremos y contamos:
for subpath in subdirectorios:
    jpegCounter = len(glob.glob1(pathimagenestrain+'/'+subpath+'/',"*.jp*g"))
    numeroimagenes[subpath] = jpegCounter

print('Número total de clases es:', len(subdirectorios))    
print('Número total de imágenes (jpg/jpeg) por clase es :{}'.format(numeroimagenes))

Guardamos en una variable el nombre da cada fichero.

In [ ]:
# Guardamos el path completo a cada fichero
ficheros = [name for name in glob.glob(pathimagenestrain+'/*/*', recursive=True)]

Cálculo y chequeo del rango dinámico, canales de color/grises y data-types. Aunque no es imprescindible, sí es una buena idea contrastar los data-types, ya que los data types unicode 8, 16 y 32 son más rápidos de procesar en Tensorflow.

In [ ]:
maximo = 0
minimo = 1024
#256
canales = 0

imagenes = []
tipos = []

# Recorremos los ficheros y vamos guardando máximos y mínimos por canal
for i in ficheros: 
    img = cv2.imread(i)
    # Extraemos max y min por colores
    max_channels = np.amax([np.amax(img[:,:,0]), np.amax(img[:,:,1]), np.amax(img[:,:,2])])
    min_channels = np.amax([np.amin(img[:,:,0]), np.amin(img[:,:,1]), np.amin(img[:,:,2])])
    if max_channels > maximo:
        maximo = max_channels
    if min_channels < minimo:
        minimo = min_channels
    # Guardamos los canales (el máximo encontrado)
    if len(img.shape) > canales:
        canales = len(img.shape)

    # Guardamos los data types que encontramos.
    if img.dtype not in tipos:
        tipos.append(img.dtype)

print("Número de imagenes leídas en /train(entrenamiento): {}\n".format(len(ficheros)))

print("Tipos de datos de los ficheros leídos en /train(entrenamiento): {}\n".format(tipos))

print("Número de canales de color de los ficheros: {}\n".format(canales))

print("Max. y Min de todas las imágenes en /train son : Max={}, Min={}\n".format(maximo, minimo))

print("El rango dinámico es: {}".format(((maximo - minimo) * 100) / 255))

### <u>Gráficos con el conteo de etiquetas y de las frecuencias relativas</u>

In [ ]:
#Gráfico inicial del conteo de etiquetas

plt.figure(figsize=(10,8))
labels = sns.barplot(x=list(numeroimagenes.keys()), y=list(numeroimagenes.values()));

# Para imprimir los valores por columnas
for i, v in enumerate(list(numeroimagenes.values())):
    #print (i, v)
    labels.text(i-0.15, v , str(v), color='darkblue', fontsize=10, fontweight='bold')
plt.title('Número de imágenes por clase', fontsize =14, color = 'darkblue');

A continuación se muestra la gráfica de las <u>frecuencias relativas</u> de las distintas etiquetas.

In [ ]:
plt.figure(figsize=(6,6))
plt.title("Frecuencias relativas en % para las clases (labels)", fontsize = 14, color = 'darkblue')
# Contamos los valores por clases y vamos calculando sus frecuencias relativas.
sorted_counts = list(np.array(list(numeroimagenes.values()))/sum(list(numeroimagenes.values())))
plt.pie(sorted_counts, labels = list(numeroimagenes.keys()), startangle = 30,
        counterclock = False, autopct='%.2f%%', textprops={'fontsize':'14'})
plt.show();

### <u>Gráfico mostrando cinco imágenes por clase</u>

In [ ]:
from PIL import Image
import random

pathimagenes = '/kaggle/input/kaggle-plant-pathology-2021-modificat/train/'
imagenes = []
etiquetas = []

for subpath in subdirectorios:
    ficheros = random.sample([name for name in glob.glob1(pathimagenes + subpath + '/',"*.jpg")],5)
    for i in ficheros:
        imagen = pathimagenes + subpath + '/' + i
        img = cv2.imread(imagen, 1)
        imagenes.append(img)
        etiquetas.append(subpath)

fig = plt.figure(figsize=(24, 24))
fig.suptitle("5 imágenes de cada clase (label)", fontsize = 24);
for i in range(len(imagenes)):
    ax = plt.subplot(6, 5, i+1)
    plt.imshow(imagenes[i])
    plt.title(etiquetas[i], fontsize = 18)
    plt.axis("off")
plt.show();

Esta parte del códico no se utiliza en nuestro problema, pero sí en el problema original de Kaggle. Se utiliza para 'binarizar' (OneHotEncoder) de MultiLabels, ya qe el problema original tienen multilabels no excluyentes.

In [ ]:
#from sklearn.preprocessing import MultiLabelBinarizer

## Convertimos el array a una lista
#s = list(p_train['labels'])
#mlb = MultiLabelBinarizer()
## Construimos un dataframe con el resultado de aplicar el MLB con las nuevas clases como columnas con el mismo índice que p_train.
#trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=p_train.index)
#listadoetiquetas = list(mlb.classes_)

## Enriquecimiento y transformación de los datos (Data Augmentation):
Generamos dos generadores de imágenes de Keras, uno para el set de entrenamiento y validación (train_datagen) y otro para el set de test. En los generadores incluimos los siguientes parámetros que enriquecerán las imágenes, lo cual beneficiará la generalización del modelo:
* Normalización
* Rotación
* Ensanchado
* Alargado
* Zoom o focalización
* Se le da la vuelta a la imagen sobre el eje de abscisas

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255., # Normalización
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    vertical_flip = False)

test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.,  # Normalización
    #rotation_range=20,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #horizontal_flip=True,
    #validation_split = 0.2,
    #zoom_range = 0.2,
    #shear_range = 0.2,
    vertical_flip = False)

Creación de los sets de entrenamiento, validación y test basados en los generadores anteriores. Nótese la división 'exacta' de las imágenes resultantes para que los batches sean uniformes.

In [ ]:
# Creamos los sets de entreno, validación y otro específico para evaluación (directorio /test y sin labels).
set_entreno = train_datagen.flow_from_directory(
    directory= '/kaggle/input/kaggle-plant-pathology-2021-modificat/train/',
    target_size=(256, 256),
    color_mode="rgb",
    class_mode="categorical",
    classes = numeroimagenes.keys(),
    batch_size=30, ## 7800/30 = 260
    shuffle=True,
    seed=27,
    save_to_dir=None,
    follow_links=False,
    subset='training',
    interpolation="nearest"
)

set_validacion = train_datagen.flow_from_directory(
    directory='/kaggle/input/kaggle-plant-pathology-2021-modificat/train/',
    target_size=(256, 256),
    color_mode="rgb",
    class_mode="categorical",
    classes = numeroimagenes.keys(),
    batch_size=30, # Divide 1950/30 a 65 batches
    shuffle=True,
    seed=27,
    save_to_dir=None,
    follow_links=False,
    subset='validation',
    interpolation="nearest",
)

# Aquí hay un pequeño 'truco'. Si pasamos como clase 'test' el generador toma esto como el subdirectorio de 'clases', aunque realmente no hay tales.
set_test = test_datagen.flow_from_directory(
    directory = '/kaggle/input/kaggle-plant-pathology-2021-modificat/',
    target_size =(256, 256),
    color_mode = "rgb",
    classes = ['test'],
    shuffle = False,
    seed = 27,
    batch_size = 39, # 39 divide al set de test perfectamente (7527/39 = 193 batches)
    class_mode = None,
)

# Este último set no implementa shuffle (shuffle=False) y se usará para las 
# predicciones y así poder comparar las clases de las predicciones vs. clases 
# verdaderas.
set_validacion_ordenado = train_datagen.flow_from_directory(
    directory='/kaggle/input/kaggle-plant-pathology-2021-modificat/train/',
    target_size=(256, 256),
    color_mode="rgb",
    class_mode="categorical",
    classes = numeroimagenes.keys(),
    batch_size=30, # Divide validación 1950/30 en 65 batches
    shuffle=False,
    seed=27,
    save_to_dir=None,
    follow_links=False,
    subset='validation',
    interpolation="nearest",
)

Guardamos las etiquetas según vienen en el conjunto de validación, sin Shuffle. Se utilizará al final para evaluar el f1-score en el entrenamiento.

In [ ]:
labelslista  = list(set_entreno.class_indices.keys())

In [ ]:
print("Orden 'real' tras la binarización (a tener en cuenta en predicciones): {}".format(labelslista))

A continuación se muestra una imagen por clase con los datos 'aumentados', es decir, tras las funciones de aumento de datos implementadas en las celdas anteriores. Esto nos da una idea de cómo se entrenarán nuestras imágenes.

In [ ]:
# Grafico con una imagen por clase tras el data augmentation.
indices = []
imagenes=[]
etiquetas = []
class_names = []

while len(imagenes) < 6:
  for i in range(len(class_names)):
    indice = np.where(labels[i] == 1)[0][0]
    #class_names = list(set_entreno.class_indices.keys())
    if ((indice in indices) == False):
      indices.append(indice)
      imagenes.append(images[i])
      etiquetas.append(class_names[indice])
      #print(etiquetas)

  images, labels = set_entreno.next()
  class_names = list(set_entreno.class_indices.keys())

fig = plt.figure(figsize=(24, 24))
for i in range(len(imagenes)):
  ax = plt.subplot(6, 6, i+1)
  plt.imshow(imagenes[i])
  plt.title(etiquetas[i])
  plt.axis("off")
fig.suptitle("Una imagen 'aumentada' de cada clase (label) tras las transformaciones", fontsize = 24);

# <u>Determinación de la estrategia de entrenamiento y validación</u>

#### Creación de las Métricas, callbacks y optimizadores. Nótese cómo vamos guardando el mejor modelo (fichero .h5 que se entrega como un activo a nuestro cliente en el repositorio público de Kaggle que incluimos con la entrega.
Además del checkpoint (guardar el mejor modelo), se implementa un Early Stopper de 5 épocas que monitoriza nuestra función f1-score en validación y también una función `ReduceOnPlateau()` que monitoriza también f1-score en validación y disminuirá el learning rate en un 10% si no se presentan mejoras en los últimas 3 épocas. De esta forma se pretende incentivar un aprendizaje más granular en estos puntos de 'no aprendizaje' para tratar de recuperar aprendizaje. 

In [ ]:
from datetime import datetime
timestampmodelo = datetime.now()
timestampmodelo = timestampmodelo.strftime("%d-%b-%Y_%H%M")

METRIC = "val_f1_score"

def create_callbacks(model, metric = METRIC):
    
    guardar_path = './mejor_modelo_' + model.name + '_' + timestampmodelo + '.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath = guardar_path,
        monitor = metric,
        mode ='max',
        save_best_only=True,
        verbose=1,
    )

    reducccionlr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = metric,
        mode = 'max',
        factor = 0.1,
        patience = 3,
        verbose = 0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor = metric,
        mode ='max',
        patience = 5, 
        verbose = 1
    )
    
    callbacks = [checkpoint, reducccionlr, earlystop]         
    
    return callbacks

Función general de compilación de los modelos, introduciendo como optimizador un descenso del gradiente con 'momentum' tipo Nesterov y como métrica la funcion f1-score 'macro'.

In [ ]:
def compile_model(model, lr=0.03):
    
    optimizer = tf.keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    metrics = [
    tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

## <u>Creación de modelos y entrenamientos iniciales: DenseNet121 y ResNet50 con/sin entrenamiento de todas las capas.</u>

### <u>Modelo DenseNet121</u>

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

seed = 37
tf.random.set_seed(seed)

base_model = tf.keras.applications.DenseNet121(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')

## Descomentar si no queremos entrenal la primera parte del DenseNet121
#for layer in base_model.layers:
#    layer.trainable = False

x = base_model.output
# fully connected layer D64 a D16 y por último a 6 clases.
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# Output final con función sigmoid (potencialmente más de una, asumimos clases no excluyentes) 
predicciones = tf.keras.layers.Dense(6, activation='softmax')(x)
modeloDenseNet121 = tf.keras.Model(base_model.inputs, predicciones, name='DenseNet121')

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    #model = create_model()
    modeloDenseNet121 = compile_model(modeloDenseNet121)
    callbacks = create_callbacks(modeloDenseNet121)
    historymodeloDenseNet121 = modeloDenseNet121.fit(
                        set_entreno,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = set_validacion,
                        verbose=VERBOSE
                       )

Podemos ver la diferencia en cuanto al número de parámetros si descomentamos la siguiente celda.

In [ ]:
#modeloDenseNet121.summary()

Recalculamos la pérdida y el f1-score por si hubiera diferencias.

In [ ]:
loss, f1score = modeloDenseNet121.evaluate(set_validacion,verbose=1)
print('Pérdida y f1-score del modelo {}: pérdida={} y f1-score(macro)={}'.format(modeloDenseNet121.name, round(loss,3), round(f1score,3)))

In [ ]:
resultadosDenseNet121 = pd.DataFrame(historymodeloDenseNet121.history)
modeloDenseNet121 = 'modeloDenseNet121_history06052021_con.csv'
with open(modeloDenseNet121, mode='w') as f:
    resultadosDenseNet121.to_csv(f)

### <u>Modelo ResNet50</u>
Se construye y prueba un modelo basado en ResNet50 con dos variantes: entrenando todo el modelo o únicamente las capas fully connected finales (D64->D16->D6).

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input

seed = 43
tf.random.set_seed(seed)

#base_model = tf.keras.applications.DenseNet121(weights=weight_path, include_top=False, pooling='avg')
base_model = tf.keras.applications.ResNet50(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')

# Descomentar para no entrenar la primera parte del ResNet50, solo las capas finales FC
#for layer in base_model.layers:
#    layer.trainable = False

x = base_model.output
# fully connected layer del Resnet a D64, a D16 y por último a D con 6 clases.
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# Output final con función softmax (asumimos clases excluyentes) 
predicciones = tf.keras.layers.Dense(6, activation='softmax')(x)
modeloResNet50 = tf.keras.Model(base_model.inputs, predicciones, name='ResNet50Prac1Final')

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

# Forzamos entrenar en la GPU
with tf.device('/device:GPU:0'):
    
    #model = create_model()
    modeloResNet50 = compile_model(modeloResNet50)
    callbacks = create_callbacks(modeloResNet50)
    historymodeloResNet50 = modeloResNet50.fit(
                        set_entreno,
                        epochs = EPOCHS,
                        callbacks = callbacks,
                        validation_data = set_validacion,
                        verbose = VERBOSE
                       )

In [ ]:
resultadosmodeloResNet50 = pd.DataFrame(historymodeloResNet50.history)
modeloResNesnet50fichero = 'modeloResNet50_history06052021_con.csv'
with open(modeloResNesnet50fichero, mode='w') as f:
    resultadosmodeloResNet50.to_csv(f)

In [ ]:
loss, f1score = modeloResNet50.evaluate(set_validacion,verbose=1)
print('La pérdida y f1-score del modelo {} en validación son: pérdida={} y f1-score(macro)={}'.format(modeloResNet50.name, round(loss,3), round(f1score,3)))

## <u>Gráficas compartivas ResNet50 vs. DenseNet121 con/sin entrenamiento de todas las capas</u>:
El siguiente gráfico muestra el resumen comparativo en cuanto a pérdida y f1-score en training y validación de ambos modelos preseleccionados y entrenados tanto con todas las capas como con las 3 últimas capas (capas fully connected añadidas: D64->D16->D6). Los resultados agrupados los tenemos en un fichero CSV dentro del repositorio de datos de la entrega: `/kaggle/input/resultados-modelos-prac1-gpu/modelos_Resultados_ResNet50_vs_DenseNet121.csv`

In [ ]:
resultados_1 = pd.read_csv('/kaggle/input/resultados-modelos-prac1-gpu/modelos_Resultados_ResNet50_vs_DenseNet121.csv')

In [ ]:
fig, ((ax1,ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2,4, figsize=(24,8))
plt.suptitle("Gráficas de pérdida y f1-score para los modelos \n (Sets entrenamientor y validación)", color='darkblue')
plt.style.use('seaborn')

ax1.plot(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['loss'], color='blue')
ax1.plot(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['val_loss'], color='orange')
ax1.set_xticks(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['epoch'])
ax1.legend(['pérdida entren.', 'perdida val.'])
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Pérdida (loss)')
ax1.set_title("Modelo ResNet50 solo últimas capas")
               
ax2.plot(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['f1_score'].to_list(),color='maroon')
ax2.plot(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['val_f1_score'].to_list(),color='lime')
ax2.set_xticks(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['epoch'].to_list())
ax2.set_xticklabels(resultados_1[resultados_1['modelo']=='ResNet50_ultimacapa']['epoch'].to_list())
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Pérdida (loss)')
ax2.legend(['f1-score entren.', 'f1-score valid.'])
ax2.set_title("Pérdida Modelo ResNet50 últimas capas")


ax3.plot(resultados_1[resultados_1['modelo']=='ResNet50_todo']['loss'].to_list(), color='blue')
ax3.plot(resultados_1[resultados_1['modelo']=='ResNet50_todo']['val_loss'].to_list(), color='orange')
ax3.set_xticks(resultados_1[resultados_1['modelo']=='ResNet50_todo']['epoch'].to_list())
ax3.set_xticklabels(resultados_1[resultados_1['modelo']=='ResNet50_todo']['epoch'].to_list())
ax3.legend(['pérdida entren.', 'perdida val.'])
ax3.set_xlabel('Epochs')
ax3.set_ylabel('Pérdida (loss)')
ax3.set_title("Modelo ResNet50 todas las capas")
               
ax4.plot(resultados_1[resultados_1['modelo']=='ResNet50_todo']['f1_score'].to_list(),color='maroon')
ax4.plot(resultados_1[resultados_1['modelo']=='ResNet50_todo']['val_f1_score'].to_list(),color='lime')
ax4.set_xticks(resultados_1[resultados_1['modelo']=='ResNet50_todo']['epoch'].to_list())
ax4.set_xticklabels(resultados_1[resultados_1['modelo']=='ResNet50_todo']['epoch'].to_list())
ax4.set_yticks(resultados_1[resultados_1['modelo']=='ResNet50_todo']['val_f1_score'].to_list())
ax4.set_xlabel('Epochs')
ax4.set_ylabel('Pérdida (loss)')
ax4.legend(['f1-score entren.', 'f1-score valid.'])
ax4.set_title("f1-score Modelo ResNet50 todas las capas")


ax5.plot(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['loss'].to_list(), color='blue')
ax5.plot(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['val_loss'].to_list(), color='orange')
ax5.set_xticks(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['epoch'].to_list())
ax5.set_xticklabels(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['epoch'].to_list())
ax5.legend(['pérdida entren.', 'perdida val.'])
ax5.set_xlabel('Epochs')
ax5.set_ylabel('Pérdida (loss)')
ax5.set_title("Modelo DenseNet50 solo últimas capas")
               
ax6.plot(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['f1_score'].to_list(),color='maroon')
ax6.plot(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['val_f1_score'].to_list(),color='lime')
ax6.set_xticks(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['epoch'].to_list())
ax6.set_xticklabels(resultados_1[resultados_1['modelo']=='DenseNet121_ultimacapa']['epoch'].to_list())
ax6.set_xlabel('Epochs')
ax6.set_ylabel('Pérdida (loss)')
ax6.legend(['f1-score entren.', 'f1-score valid.'])
ax6.set_title("f1-score Modelo DesNet121 (últimas capas)")


ax7.plot(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['loss'].to_list(), color='blue')
ax7.plot(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['val_loss'].to_list(), color='orange')
ax7.set_xticks(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['epoch'].to_list())
ax7.legend(['pérdida entren.', 'perdida val.'])
ax7.set_xlabel('Epochs')
ax7.set_ylabel('Pérdida (loss)')
ax7.set_title("Modelo DenseNet121 todas las capas")
               
ax8.plot(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['f1_score'].to_list(),color='maroon')
ax8.plot(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['val_f1_score'].to_list(),color='lime')
ax8.set_xticks(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['epoch'].to_list())
ax8.set_xticklabels(resultados_1[resultados_1['modelo']=='DenseNet121_todo']['epoch'].to_list())
ax8.set_xlabel('Epochs')
ax8.set_ylabel('Pérdida (loss)')
ax8.legend(['f1-score entren.', 'f1-score valid.'])
ax8.set_title("f1-score Modelo DenseNet121 todas las capas")

plt.tight_layout()
plt.show();

## <u>Selección y entrenamiento final del modelo ResNet50->D64->D16-D6 elegido</u>

A continuación, ya elegido el modelo ResNet50 entrenado en su totalidad,, construimos una función que lo compilará e incluirá las métricas adicionales establecidas con nuestro cliente: f1-score, accuracy, precision y recall. Además, estas métricas facilitarán la construcción de la matriz de confusión resultante de nuestro entrenamiento.

In [ ]:
def compile_model_final(model, learning_rate=0.03):
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.03, decay=1e-4, momentum=0.8, nesterov=True)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    # tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
        tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score"),       
        #keras.metrics.TruePositives(name='tp'),
        #keras.metrics.FalsePositives(name='fp'),
        #keras.metrics.TrueNegatives(name='tn'),
        #keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.CategoricalAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

Entrenamiento final del ResNet50 con 30 épocas y las nuevas métricas.

In [ ]:
EPOCHS= 30
VERBOSE =1

tf.keras.backend.clear_session()

# Forzamos entrenar en la GPU
with tf.device('/device:GPU:0'):
    
    modeloResNet50 = compile_model_final(modeloResNet50)
    callbacks = create_callbacks(modeloResNet50)
    historymodeloResNet50 = modeloResNet50.fit(
                        set_entreno,
                        epochs = EPOCHS,
                        callbacks = callbacks,
                        validation_data = set_validacion,
                        verbose = VERBOSE
                       )

## <u>Predicciones sobre el conjunto de imágenes de test y creación del fichero test.csv</u>
Calculamos las predicciones de nuestro set de test que hemos creado al principio. Lo podemos hacer si hemos ejecutado la celda anterior (entrenando 30 épocas) o también si subimos el modelo que hemos guardado previamente en el repositorio Kaggle: '/kaggle/input/resultados-modelos-prac1-gpu/mejor_modelo_ResNet50Prac1Final.h5'

In [ ]:
# Descomentar esta celda si queremos subir y utilizar el modelo **ya entrenado** con un f1-score de 0.93409
#from keras.models import load_model

#modeloResNet50 = load_model('/kaggle/input/resultados-modelos-prac1-gpu/mejor_modelo_ResNet50Prac1Final_06052021.h5')
##modeloResNet50 = load_model('/kaggle/input/resultados-modelos-prac1-gpu/mejor_modelo_ResNet50Prac1Final_05-Jun-2021_1240.h5')

Se efectúan las predicciones del modelo sobre las imágenes de test (set_test).

In [ ]:
set_test.reset()
predicciones = modeloResNet50.predict(set_test)

Tomamos nota del orden en el que vienen las etiquetas en el generador de imágenes del conjunto de entrenamiento-validación ya que la predicción se efectuará de acuerdo a este orden.

In [ ]:
print('Orden de las etiquetas: {}'.format(set_entreno.class_indices.keys()))
#set_test.class_indices.keys()

In [ ]:
print('Orden de las etiquetas: {}'.format(set_validacion_ordenado.class_indices.keys()))
#set_test.class_indices.keys()

In [ ]:
# Para confirmar
#set_entreno.class_indices.items()

Guardamos en un array las predicciones con una mayor probabilidad en cada imagen del conjunto de test.

In [ ]:
preds_clases_indices = predicciones.argmax(axis=-1)

Eliminamos el prefijo  'test/' del nombre de los ficheros.

In [ ]:
ficheros_test = [str(i).replace('test/', '')  for i in set_test.filenames]

Construimos una tupla (ficheros, clase predicha), a partir de los arrays anteriores. Una vez construido, lo guardamos en un DataFrame y lo exportamos como test.csv.

In [ ]:
indices_de_clases = {v: k for k, v in set_entreno.class_indices.items()}
preds_clases = np.vectorize(indices_de_clases.get)(preds_clases_indices)
filenames_to_clases = list(zip(ficheros_test, preds_clases))

Resumen (conteo) de las predicciones por clases:

In [ ]:
from collections import Counter
print("conteo de predicciones: {}".format(Counter(preds_clases)))

Obtenemos una gráfica de nuestras predicciones en porcentaje (frecuencias relativas).

In [ ]:
# Grafica del % de etiquetas en las predicciones.
plt.figure(figsize=(6,6))
plt.title("Frecuencias relativas en % de las predicciones", fontsize = 14, color = 'darkblue')
# Contamos los valores por clases y vamos calculando sus frecuencias relativas.
sorted_counts_pred = list(np.array(list(Counter(preds_clases).values()))/sum(list(Counter(preds_clases).values())))
plt.pie(sorted_counts_pred, labels = list(Counter(preds_clases).keys()), startangle = 30,
        counterclock = False, autopct='%.2f%%', textprops={'fontsize':'14'})
plt.show();

Observamos que hay un mayor número de imágenes con enfermedades respecto a las sanas (C0), lo cual no está muy en línea con los sets de entrenamiento y validación. En cualquier caso es posible que el set de test sea completamente diferente en cuanto a sus clases.

In [ ]:
prediccionesResNet50 = pd.DataFrame(filenames_to_clases)
predicciones_fichero = 'test.csv'
with open(predicciones_fichero, mode='w') as f:
    prediccionesResNet50.to_csv(f)

Por últmos obtenemos una imagen por clase predicha y la visualizamos para 'comprobar' los resultados.

In [ ]:
path_test = '/kaggle/input/kaggle-plant-pathology-2021-modificat/test/'

# Grafico con una imagen por clase tras la  prediccion.
indices = []
imagenes=[]
etiquetas = []
class_names = []

while len(imagenes) < 6:
    for i in filenames_to_clases:
    
        etiqueta = i[1]
        if ((etiqueta in etiquetas) == False):
            imagenes.append(path_test+str(i[0]))
            etiquetas.append(i[1])
            #print(etiquetas)

    #images = set_test.next() 
    #labels = preds_clases_indices
  
fig = plt.figure(figsize=(24, 24))
for i in range(len(imagenes)):
    ax = plt.subplot(6, 6, i+1)
    img = cv2.imread(imagenes[i], 1)
    plt.imshow(img)
    plt.title('Clase predicha: ' + etiquetas[i])
    plt.axis("off")
fig.suptitle("Una imagen predicha de cada clase (label) con nuestro modelo ResNet50->D64->D16->D6", fontsize = 24);
fig.tight_layout()

Calculamos las predicciones en el conjunto de validación para corroborar el f1-score, el cual calculamos nuevamente.

In [ ]:
predicciones_val_1 = modeloResNet50.predict(set_validacion_ordenado)
predicciones_val = np.argmax(predicciones_val_1, axis=-1)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("Precision f1-score 'macro': {}".format(precision_score(set_validacion_ordenado.classes, predicciones_val , average="macro")))

## <u>Gráficas del modelo elegido: Accuracy, f1-score, precision y recall</u>
A continuación se obtienen las gráficas que incluyen accuracy, f1-score, precision y recall <u>**ponderadas**</u> (`average = 'weighted'`) del entrenamiento del modelo ResNet50 elegido.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

# Se calculan las métricas al final para el conjunto de validación
print('Precisión (ponderada=weighted) en el conjunto de validación: {}'.format(precision_score(set_validacion_ordenado.classes, predicciones_val , average="weighted")))
print('Recall (ponderada=weighted) en el conjunto de validación:    {}'.format(recall_score(set_validacion_ordenado.classes, predicciones_val , average="weighted")))
print('f1-score (ponderada=weighted) en el conjunto de validación:  {}'.format(f1_score(set_validacion_ordenado.classes, predicciones_val , average="weighted")))

In [ ]:
#fig, ((ax1,ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2,4, figsize=(24,8))
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3, figsize=(18,6))
plt.suptitle("Gráficas de pérdida, f1-score, accuracy, recall y precision \n para el modelo ResNet50->D64->D16->D6 (entren. y valid.)", color='darkblue')
plt.style.use('seaborn')

ax1.plot(historymodeloResNet50.history['loss'], color='blue')
ax1.plot(historymodeloResNet50.history['val_loss'], color='orange')
ax1.set_xticks(np.arange(len(historymodeloResNet50.history['loss'])))
ax1.legend(['pérdida entren.', 'perdida val.'])
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Pérdida (loss)')
ax1.set_title("Pérdida Modelo ResNet50 final D64->D16->D6")

ax2.plot(historymodeloResNet50.history['f1_score'], color='maroon')
ax2.plot(historymodeloResNet50.history['val_f1_score'], color='lime')
ax2.set_xticks(np.arange(len(historymodeloResNet50.history['loss'])))
ax2.legend(['f1-score entren.', 'f1-score valid.'])
ax2.set_xlabel('Epochs')
ax2.set_ylabel('F1-Score')
ax2.set_title("f1-score(macro) Modelo ResNet50 final D64->D16->D6")

ax3.plot(historymodeloResNet50.history['accuracy'], color='purple')
ax3.plot(historymodeloResNet50.history['val_accuracy'], color='red')
ax3.set_xticks(np.arange(len(historymodeloResNet50.history['loss'])))
ax3.legend(['Accuracy entren.', 'Accuracy valid.'])
ax3.set_xlabel('Epochs')
ax3.set_ylabel('Accuracy')
ax3.set_title("Accuracy (cross-entropy) Modelo ResNet50 final D64->D16->D6")

ax4.plot(historymodeloResNet50.history['precision'], color='grey')
ax4.plot(historymodeloResNet50.history['val_precision'], color='green')
ax4.set_xticks(np.arange(len(historymodeloResNet50.history['loss'])))
ax4.legend(['Precision entren.', 'Precision valid.'])
ax4.set_xlabel('Epochs')
ax4.set_ylabel('Precision')
ax4.set_title("Precision Modelo ResNet50 final D64->D16->D6")

ax5.plot(historymodeloResNet50.history['recall'], color='gold')
ax5.plot(historymodeloResNet50.history['val_recall'], color='darkblue')
ax5.set_xticks(np.arange(len(historymodeloResNet50.history['loss'])))
ax5.legend(['Recall entren.', 'Recall valid.'])
ax5.set_xlabel('Epochs')
ax5.set_ylabel('Recall')
ax5.set_title("Recall Modelo ResNet50 final D64->D16->D6")

plt.tight_layout()
plt.show();

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, recall_score, multilabel_confusion_matrix

clases_val_ord = set_validacion_ordenado.class_indices.keys()
# Calculamos la matriz de confusion

cfmatrix  = confusion_matrix(set_validacion_ordenado.classes, predicciones_val)
cfmatrixmulti = multilabel_confusion_matrix(set_validacion_ordenado.classes, predicciones_val)

In [ ]:
#Graficamos con seaborn y un heatmap para visualizar
import seaborn as sns
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = "12"
sns.heatmap(cfmatrix, annot=True, cmap="BuPu", fmt=".0f", xticklabels=clases_val_ord, yticklabels=clases_val_ord)
plt.tight_layout()
plt.title("Matriz de confusión modelo RestNet50: \n (RestNet50->D64->D16->D6)\n")
plt.ylabel('Clase verdadera')
plt.xlabel('Clase predicha')
plt.show();

In [ ]:
print('Classification Report/Informe sobre la clasificación de acuerdo a las 6 enfermedades del manzano')
print(classification_report(set_validacion_ordenado.classes, predicciones_val,target_names=clases_val_ord))

# <u>Apéndice</u>: Modelos entrenados en el problema original Kaggle y en nuestro problema para la selección previa de modelos a utilizar.
A continuación se presentan los 6 modelos probados tanto para el desafío original en Kaggle como para nuestro problema. Los modelos son:  
1. Resnet50v2  
2. InceptionV3  
3. EfficientNetB0  
4. Resnet50  
5. IncepcionResNet50V2  
6. DenseNet121  
 
Para estos modelos dentro del desafío original, es necesario construir unos generadores algo diferentes que toman como input los subdirectorios de las imágenes reducidas en tamaño (256), ya que si utilizamos las imágenes originales éstas son demasiado grandes en tamaño y el entrenamiento se ralentiza mucho. En cualquier caso estos generadores requieren de un dataframe diferente, donde las clases se consideran multilabels y la binarizacion (OneHotEncoder) se implementa con un método diferente MultiLabelBinarizer() de sklearn.preprocessing.


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

train_data = train_datagen.flow_from_dataframe(
    df_entreno2,
    directory= '/kaggle/input/resized-plant2021/img_sz_256/', # originales en 'plant-pathology-2021-fgvc8/train_images/'
    x_col = "image",
    y_col=  labelslista,
    color_mode="rgb",
    #target_size = (256,256),
    #class_mode="categorical",
    batch_size=32,
    subset = "training",
    shuffle=True,
    seed=27,
    class_mode='raw'
)

validacion_data = train_datagen.flow_from_dataframe(
    df_entreno2,
    directory= '/kaggle/input/resized-plant2021/img_sz_256/', # originales en 'plant-pathology-2021-fgvc8/train_images/'
    x_col = "image",
    y_col=  labelslista,
    color_mode="rgb",
    #target_size = (256,256),
    #class_mode="categorical",
    batch_size=32,
    subset = "validation",
    shuffle=True,
    seed=27,
    class_mode='raw'
)

## <u>Construcción y entrenamiento de 6 modelos (preselección)</u>:
Se construyen y entrenan los siguientes modelos para determinar cuáles de ellos son más prometedores respecto a la métrica f1-score(macro) utilizada:  

    1.-Resnet50v2  
    2.-InceptionV3  
    3.-EfficientNetB0  
    4.-Resnet50  
    5.-IncepcionResNet50V2  
    6.-DenseNet121  

### <u>Modelo ResNet50V2:</u>

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.models import Sequential

def create_model():
    
    pretrained = ResNet50V2(include_top=False, weights='imagenet',input_shape=[256,256, 3])
    # pretrained = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=[256, 256, 3])
            
    x = pretrained.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(6, activation="sigmoid", dtype='float32')(x)
        
    model = tf.keras.Model(pretrained.input, outputs, name='modeloResNetV2')
    return model

model = create_model()

In [ ]:
model.name

In [ ]:
# Función que compilará el modelo con Adam y la learning rate como parámetro. 
# También introducimos nuestra medidad como F1-score 'macro'

def compile_model(model, lr=0.03):
    
    optimizer = tf.keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    # tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

Los siguientes parámetros son comunes a todos los modelos. Introducen un 'checkpoint' para guardar los mejores modelos según se van obteniendo máximos en nuestra función (f1-score en validación). También definimos un EarlyStop con 5 epochs de paciencia y un parámetro que actúa como reductor del learning rate si se producen situaciones de estancamiento ([`ReduceLROnPlateau()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau))

In [ ]:
METRIC = "val_f1_score"

def create_callbacks(metric = METRIC):
    
    guardar_path = './mejor_modelo_' + model.name + '_' + timestampmodelo + '.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath = guardar_path,
        monitor = metric,
        mode ='max',
        save_best_only=True,
        verbose=1,
    )

    reducccionlr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = metric,
        mode = 'max',
        factor = 0.1,
        patience = 3,
        verbose = 0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor = metric,
        mode ='max',
        patience = 5, 
        verbose = 1
    )
    
    callbacks = [checkpoint, reducccionlr, earlystop]         
    
    return callbacks

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    model = create_model()
    model = compile_model(model, lr=0.001)
   
    callbacks = create_callbacks()
    
    history = model.fit(
                        train_data,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = validacion_data,
                        verbose=VERBOSE
                       )

In [ ]:
resultados = pd.DataFrame(history.history)

In [ ]:
modeloResNetV2 = 'modeloResNetV2_history.csv'
with open(modeloResNetV2, mode='w') as f:
    resultados.to_csv(f)

### <u>Modelo EfficientNet80 (problema original):</u>

Aquí utilizamos tanto los datos de nuestro caso como los **datos originales** del desafío Kaggle `/kaggle/input/plant-pathology-2021-fgvc8/train.csv'` para ir probando diferentes modelos y su performance e idoneidad para nuestro problema.

In [ ]:
p_train = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')

Individualizamos cada etiqueta de cada imagen, ya que cada una trae varias.

In [ ]:
p_train['labels'] = p_train['labels'].apply(lambda string: string.split(' '))

Para la construcción de este dataframe (df_entreno) hay que referirse a un segundo [notebook](https://www.kaggle.com/nameeman/practica-1-dl-galech-amillano-probl-originalkaggle/edit) donde se trata el problema Kaggle original (multiclase-multilabel).  
https://www.kaggle.com/nameeman/practica-1-dl-galech-amillano-probl-originalkaggle/edit

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

train_data = train_datagen.flow_from_dataframe(
    df_entreno,
    directory= '/kaggle/input/resized-plant2021/img_sz_256/', # originales en 'plant-pathology-2021-fgvc8/train_images/'
    x_col = "image",
    y_col=  "labels",
    color_mode="rgb",
    #target_size = (256,256),
    #class_mode="categorical",
    batch_size=32,
    subset = "training",
    shuffle=True,
    seed=27,
    class_mode='raw'
)

validacion_data = train_datagen.flow_from_dataframe(
    df_entreno,
    directory = '/kaggle/input/resized-plant2021/img_sz_256/',
    x_col = "image",
    y_col = "labels",
    color_mode="rgb",
    #target_size = (256,256),
    #class_mode="categorical",
    batch_size=32,
    shuffle=True,
    subset = "validation",
    seed=27,
    class_mode='raw'
)

# Este último set no implementa shuffle (shuffle=False) y se usará para las 
# predicciones y así poder comparar las clases de las predicciones vs. clases 
# verdaderas.
set_validacion_ordenado = train_datagen.flow_from_dataframe(
    df_entreno,
    directory = '/kaggle/input/resized-plant2021/img_sz_256/',
    x_col="image",
    y_col = "labels",
    #target_size=(256, 256),
    color_mode="rgb",
    #class_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=27,
    subset='validation',
    interpolation="nearest",
    class_mode='raw'
)

set_test = test_datagen.flow_from_directory(
    directory = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/',
    batch_size=1,
    # No tenemos y_col porque son las que tenemos que predecir
    target_size=(256, 256),
    color_mode="rgb",
    class_mode=None,
    shuffle=False,
    seed=27,
#    interpolation="nearest",
)

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

def create_EffNetB0model():
    
    pretrained = EfficientNetB0(include_top=False, weights='imagenet',input_shape=[256, 256, 3], pooling = 'avg')
    x = pretrained.output
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    # finally, the softmax for the classifier 
    #predictions = tf.keras.layers.Dense(6, activation='sigmoid')(x)
    
    outputs = tf.keras.layers.Dense(6,activation="sigmoid", dtype='float32')(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloEffNetB0')
    return model

model = create_EffNetB0model()


def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.BinaryCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    #model = create_model()
    modeloEffNetB0 = compile_model(model)
    callbacks = create_callbacks(modeloEffNetB0)
    historymodeloDenseNet121 = modeloEffNetB0.fit(
                        set_entreno,
                        epochs = EPOCHS,
                        callbacks = callbacks,
                        validation_data = set_validacion,
                        verbose = VERBOSE
                       )

## <u>Modelo InceptionV3:</u>

In [ ]:
try: # TPUs, si tenemos habilitado el cluster y generamos archivos tipo .tfrec
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)

except ValueError: # GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # una CPU y una GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

def create_InceptionV3():
    
    pretrained = InceptionV3(include_top=False, weights='imagenet',input_shape=[256, 256, 3], pooling = 'avg')
    x = pretrained.output
    #x = tf.keras.layers.GlobalAveragePooling2D()(x) # No necesitamos pooling porque lo ponemos en la salida del EfficientNetB0
    outputs = tf.keras.layers.Dense(len(validacion_data.class_indices),activation="sigmoid", dtype='float32')(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloInceptionV3')
    return model

model = create_InceptionV3()


def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = len(validacion_data.class_indices), average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
model.name

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    #model = create_model()
    model = compile_model(model, lr=0.0001)
   
    callbacks = create_callbacks()
    
    historyIncepcionV3 = model.fit(
                        train_data,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = validacion_data,
                        verbose=VERBOSE
                       )

In [ ]:
resultadosInceptionV3 = pd.DataFrame(historyIncepcionV3.history)
modeloInceptionV3_cvs_file = 'resultadosInceptionV3_history.csv'
with open(modeloInceptionV3_cvs_file, mode='w') as f:
    resultadosInceptionV3.to_csv(f)

## <u>Modelo EfficientNetB0:</u>

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

def create_EffNetB0model():
    
    pretrained = EfficientNetB0(include_top=False, weights='imagenet',input_shape=[256, 256, 3], pooling = 'avg')
    x = pretrained.output
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    
    #x = tf.keras.layers.GlobalAveragePooling2D()(x) # No necesitamos pooling porque lo ponemos en la salida del EfficientNetB0
    outputs = tf.keras.layers.Dense(6,activation="sigmoid")(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloEffNetB0')
    return model

model = create_EffNetB0model()

In [ ]:
EPOCHS = 10
VERBOSE = 1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    #model = create_EffNetB0model()
    model = compile_model(model)
 
    callbacks = create_callbacks(model)
    
    historyModeloEffNetB0 = model.fit(
                    set_entreno,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = set_validacion,
                    verbose = VERBOSE
                   )

In [ ]:
resultadosEffNetB0 = pd.DataFrame(historyModeloEffNetB0.history)
resultadosEffNetB0_csv_file = 'resultadosEffNetB0_history2.csv'
with open(resultadosEffNetB0_csv_file, mode='w') as f:
    resultadosEffNetB0.to_csv(f)

## <u>Modelo RestNet50:</u> 
Creamos y entrenamos el modelo RestNet50 con el mismo set de datos, mismos parámetros y guardamos tanto el modelo como los resultados del entrenamiento.

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input

def create_RestNet50():
    
    pretrained = ResNet50(include_top=False, weights='imagenet',input_shape=[256, 256, 3])
    x = pretrained.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x) 
    # Para RestNet50 debemos usar softmax si utilizamos los pesos pre-entrenados.
    outputs = tf.keras.layers.Dense(6,activation="sigmoid", dtype='float32')(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloRestNet50')
    return model

model = create_RestNet50()


def compile_model(model):
    
    #optimizer = tf.keras.optimizers.Adam(lr=lr)
    optimizer = tf.keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True)

    
    loss = tf.keras.losses.BinaryCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

Entrenamos el modelo RestNet50

In [ ]:
EPOCHS = 10
VERBOSE = 1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    callbacks = create_callbacks()
    model = compile_model(model)
    historyResNet50 = model.fit(
                    train_data,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = validacion_data,
                    verbose = VERBOSE
                   )

In [ ]:
resultadosRestNet50 = pd.DataFrame(historyResNet50.history)
modeloResNet50 = 'modeloResNet50_history2.csv'
with open(modeloResNet50, mode='w') as f:
    resultadosRestNet50.to_csv(f)

## <u>Modelo DenseNet121:</u> 
Creamos y entrenamos el modelo DenseNet121 con el mismo set de datos, mismos parámetros y guardamos tanto el modelo como los resultados del entrenamiento.

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
#tf.keras.applications.densenet.DenseNet121

seed = 1200
tf.random.set_seed(seed)

#base_model = tf.keras.applications.DenseNet121(weights=weight_path, include_top=False, pooling='avg')
base_model = tf.keras.applications.DenseNet121(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')
#pesos_entrenados = '../input/keras-pretrained-models/'
#model = keras.applications.InceptionResNetV2(weights=pesos_entrenados, include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False

x = base_model.output
#fully connected layer
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
predictions = tf.keras.layers.Dense(6, activation='sigmoid')(x)
modeloDenseNet121 = tf.keras.Model(base_model.inputs, predictions, name='DenseNet121')


# <u>Gráficas del entrenamiento de los modelos</u>:

Se muestra a continuación la evolución de la pérdida y de F1-Score para los conjuntos de entrenamiento y de validación para nuestros cuatro modelos.

In [ ]:
resultados = pd.read_csv('/kaggle/input/resultados-modelos-prac1-gpu/modelos_Resultados_original_Kaggle.csv')

In [ ]:
modelos = list(resultados.modelo.unique())

fig = plt.figure(figsize=(18, 4))
fig.suptitle("Pérdida en los 6 modelos", fontsize = 20);
plt.style.use('seaborn')

for i, j in enumerate(modelos):
    ax = plt.subplot(1, 6, i+1)
    ax.plot(resultados[resultados['modelo']==str(j)]['loss'].to_list(), color='blue')
    ax.plot(resultados[resultados['modelo']==str(j)]['val_loss'].to_list(), color='orange')
    ax.set_xticks(resultados[resultados['modelo']==str(j)]['epoch'].to_list())
    ax.set_xticklabels(resultados[resultados['modelo']==str(j)]['epoch'].to_list())
    ax.legend(['pérdida entren.', 'perdida val.'])
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Pérdida (loss)')
    ax.set_title("Pérdida Modelo "+str(j))
plt.tight_layout()
plt.show();

fig2 = plt.figure(figsize=(18, 4))
fig2.suptitle("f1-score en los 6 modelos", fontsize = 20);
plt.style.use('seaborn')

for i, j in enumerate(modelos):

    ax = plt.subplot(1, 6, i+1)
    ax.plot(resultados[resultados['modelo']==str(j)]['f1_score'].to_list(), color='maroon')
    ax.plot(resultados[resultados['modelo']==str(j)]['val_f1_score'].to_list(), color='lime')
    ax.set_xticks(resultados[resultados['modelo']==str(j)]['epoch'].to_list())
    ax.set_xticklabels(resultados[resultados['modelo']==str(j)]['epoch'].to_list())
    ax.legend(['f1-score entren.', 'f1-score val.'])
    ax.set_xlabel('Epochs')
    ax.set_ylabel('f1-score (macro)')
    ax.set_title("F1-score Modelo "+str(j))

plt.tight_layout()
plt.show();

### <u> Modelo InceptionResNetv2</u>

In [ ]:
seed = 1200
tf.random.set_seed(seed)

pesos_entrenados = '../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
model = keras.applications.InceptionResNetV2(weights=pesos_entrenados, include_top=False, input_shape=(256, 256, 3))


In [ ]:
modelo_sobre_IncRSNet50V2 = tf.keras.Sequential([
    model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(6, kernel_initializer=keras.initializers.RandomUniform(seed=seed),
                        bias_initializer=keras.initializers.Zeros(), name='dense_top', activation='sigmoid')
])

# Freezing the weights
for layer in modelo_sobre_IncRSNet50V2.layers[:-1]:
    layer.trainable=False
    
modelo_sobre_IncRSNet50V2.summary()

In [ ]:
EPOCHS = 10
VERBOSE = 1
f1 = tfa.metrics.F1Score(num_classes=6, average='macro')

#callbacks = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)


modelo_sobre_IncRSNet50V2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True), 
              metrics= [f1])

with tf.device('/device:GPU:0'):
    
    callbacks = create_callbacks(modelo_sobre_IncRSNet50V2)
    #model = compile_model(model, lr=0.0001)
    history_IncRSNet50V2 = modelo_sobre_IncRSNet50V2.fit(
                    set_entreno,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = set_validacion,
                    verbose = VERBOSE
                   )

#modelo_sobre_IncRSNet50V2.fit(train_data, epochs=20, callbacks=callbacks)

In [ ]:
resultadosIncepRSNet50V2 = pd.DataFrame(history_IncRSNet50V2.history)
modeloIncepResNet50v2 = 'modeloIncepRSNet50V2_history.csv'
with open(modeloIncepResNet50v2, mode='w') as f:
    resultadosIncepRSNet50V2.to_csv(f)

## Modelo DenseNet121

In [ ]:
# Probamos un DenseNet121
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
#tf.keras.applications.densenet.DenseNet121

seed = 43
tf.random.set_seed(seed)

#base_model = tf.keras.applications.DenseNet121(weights=weight_path, include_top=False, pooling='avg')
base_model = tf.keras.applications.DenseNet121(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')
x = base_model.output
#fully connected layer
#x = tf.keras.layers.Dense(64, activation='relu')(x)
#x = tf.keras.layers.Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
output = tf.keras.layers.Dense(6, activation='sigmoid')(x)
modeloDenseNet121orig = tf.keras.Model(base_model.inputs, output, name='DenseNet121')


In [ ]:
EPOCHS = 10
VERBOSE = 1
f1 = tfa.metrics.F1Score(num_classes=6, average='macro')

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):

    callbacks = create_callbacks(modeloDenseNet121orig)
    modeloDenseNet121orig = compile_model(modeloDenseNet121orig)
    history_DenseNet121 = modeloDenseNet121orig.fit(
                    #train_data,
                    set_entreno,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = set_validacion, 
                    #validation_data = validacion_data,
                    verbose = VERBOSE
                   )

In [ ]:
resultadosDenseNet121= pd.DataFrame(history_DenseNet121.history)
modeloDenseNet121 = 'modeloDenseNet121_history2.csv'
with open(modeloDenseNet121, mode='w') as f:
    resultadosDenseNet121.to_csv(f)

In [ ]:
test_generator = test_datagen.flow_from_directory()

In [ ]:
test_generator.reset()
predicciones = model.predict_generator(test_generator, verbose=1)